In [1]:
import gurobipy as gp
from gurobipy import GRB

# Data
orders = [
    (3, 3, 99), (9, 1, 95), (15, 2, 92), (5, 5, 90), (8, 1, 86), 
    (7, 3, 81), (19, 5, 80), (30, 5, 75), (23, 1, 72), (2, 5, 70), 
    (12, 4, 70), (21, 1, 69), (26, 2, 67), (14, 4, 64), (29, 2, 64), 
    (22, 3, 62), (10, 2, 60), (17, 2, 60), (16, 1, 55), (13, 2, 53), 
    (27, 2, 52), (24, 2, 41), (1, 2, 40), (18, 4, 38), (25, 5, 38), 
    (11, 4, 37), (28, 4, 37), (6, 1, 35), (20, 5, 34), (4, 5, 32)
]

n = len(orders)
positions = range(n)

# Extract processing times and weights
TP = [order[1] for order in orders]
w = [order[2] for order in orders]

# Create a new model
m = gp.Model("weighted_completion_time")

# Create variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")

# Objective function
completion_times = [sum(TP[j] * x[j, i] for j in positions) for i in positions]
weighted_completion_time = gp.quicksum(w[i] * completion_times[i] for i in positions)
m.setObjective(weighted_completion_time, GRB.MINIMIZE)

# Constraints
# Each order must be assigned to exactly one position
m.addConstrs((gp.quicksum(x[i, j] for j in positions) == 1 for i in positions), name="order_assigned")

# Each position must be assigned exactly one order
m.addConstrs((gp.quicksum(x[i, j] for i in positions) == 1 for j in positions), name="position_filled")

# Optimize model
m.optimize()

# Display the results
if m.status == GRB.OPTIMAL:
    print("Optimal total weighted completion time:", m.objVal)
    for j in positions:
        for i in positions:
            if x[i, j].x > 0.5:
                print(f"Order {orders[i][0]} is scheduled at position {j+1} with completion time {sum(TP[k] for k in range(j+1))}")
                #print(f"Order {orders[i][0]} is scheduled at position {j+1} with completion time {sum(TP[k] for k in range(j+1))}")

else:
    print("No optimal solution found.")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 7840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 60 rows, 900 columns and 1800 nonzeros
Model fingerprint: 0x2b818745
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5436.0000000
Presolve time: 0.03s
Presolved: 60 rows, 900 columns, 1800 nonzeros
Variable types: 0 continuous, 900 integer (900 binary)

Root relaxation: objective 4.570000e+03, 268 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf